In [1]:
import numpy as np
import pandas as pd
import math
import os
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [2]:
# context = pd.read_csv('..//data/preprocessed//context_1217_1230.csv')
context = pd.read_csv('/home/jovyan/df-smart-channel/graph/data/preprocessed/context_1217_0131.csv')

In [3]:
le = LabelEncoder()
context.cust_no = le.fit_transform(context.cust_no)
context.item_id = le.fit_transform(context.item_id)

In [4]:
user_list = context.cust_no.unique()
item_list = context.item_id.unique()

In [5]:
res_df = {}
for i, item in enumerate(item_list):
    subtags_list = np.random.choice([i for i in range(0,8)], 3, replace=False).tolist()
    subtags_list.sort()
    res_df[item] = subtags_list

In [6]:
res_df

{1: [0, 2, 5],
 9: [0, 3, 5],
 13: [4, 5, 7],
 5: [1, 4, 5],
 14: [0, 4, 6],
 10: [1, 4, 7],
 8: [1, 6, 7],
 16: [2, 3, 4],
 17: [0, 1, 4],
 2: [1, 2, 3],
 7: [0, 1, 7],
 3: [2, 4, 7],
 18: [0, 1, 4],
 19: [0, 3, 7],
 11: [2, 4, 6],
 4: [0, 1, 7],
 12: [1, 4, 5],
 0: [1, 2, 3],
 15: [2, 5, 7],
 6: [2, 3, 5]}

In [7]:
# write to txt file
def export_file(date, pairs_dict_train, pairs_dict_test):
    dir_name = f'date={date}'
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
    ## train
    file_path = os.path.join(dir_name, 'item_subtag.txt')
    file = open(file_path,"w")
    print(f'write to file path: {file}')
    counts = 0
    for key,value in pairs_dict_train.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                else:
                    file.write(' %s' % int(item))
        file.write('\n')
        counts += 1
    print(counts)
    file.close()
#     ## test
#     file_path = os.path.join(dir_name, 'test_user_subtag.txt')
#     file = open(file_path,"w")
#     counts = 0
#     print(f'write to file path: {file}')
    
#     for key,value in pairs_dict_test.items():
#         file.write('%s' % key)
#         # for item in value:
#         #     file.write('%s ' % item)
#         for i, item in enumerate(value):
#             if math.isnan(item):
#                 continue
#             else:
#                 if i == (len(value)-1):
#                     file.write(' %s' % int(item))
#                     file.write('\n')
#                 else:
#                     file.write(' %s' % int(item))
#         counts += 1
#     print(counts)
    file.close()

In [9]:
test_date = 20211226

In [10]:
export_file(date = str(test_date), pairs_dict_train = res_df, pairs_dict_test = res_df )

write to file path: <_io.TextIOWrapper name='date=20211226/item_subtag.txt' mode='w' encoding='UTF-8'>
20


## Splitting

In [6]:
train_start = 20211217
train_end = 20211225
test_end = 20211226
test_date = test_end

In [7]:
## old version
# def splitting(user_list, train_start, test_date):
#     context_train = context_click[(context_click.date >= train_start) & (context_click.date < test_date)][['cust_no', 'item_id', 'show', 'click']]
#     context_test = context_click[context_click.date == test_date][['cust_no', 'item_id', 'show', 'click']]
    
#     cust_item_pair_train = context_train.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
#     cust_item_pair_test = context_test.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
    
#     # cust_item_pair_train = cust_item_pair_train[cust_item_pair_train.click > 0]
#     # cust_item_pair_test = cust_item_pair_test[cust_item_pair_test.click > 0]
    
#     trainUniqueUsers = set(cust_item_pair_train.cust_no.unique())
#     # testUniqueUsers = set(cust_item_pair_test.cust_no.unique())
#     cust_item_pair_test = cust_item_pair_test[cust_item_pair_test.cust_no.isin(trainUniqueUsers)]
    
#     items_by_cust_train = cust_item_pair_train.groupby('cust_no')['item_id'].unique()
#     items_by_cust_test = cust_item_pair_test.groupby('cust_no')['item_id'].unique()
    
#     cust_items_dict_train = dict(items_by_cust_train.apply(list))
#     cust_items_dict_test = dict(items_by_cust_test.apply(list))
#     return cust_items_dict_train, cust_items_dict_test

In [8]:
user_list = context
train_start = 20211217
test_date = 20211226

In [15]:
context_train = context[(context.date >= train_start) & (context.date < test_date)][['cust_no', 'item_id', 'show','click']]
context_test = context[context.date == test_date][['cust_no', 'item_id','click']]
context_test = context_test[context_test.cust_no.isin(trainUniqueUsers)]
trainUniqueUsers = set(context_train.cust_no.unique())
testUniqueUsers = set(context_test.cust_no.unique())

In [39]:
cust_item_pair_train = context_train.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
cust_item_pair_test = context_test.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
cust_item_pair_train = cust_item_pair_train[cust_item_pair_train.click>0]
cust_item_pair_test = cust_item_pair_test[cust_item_pair_test.click>0]

In [ ]:
train_ = pd.DataFrame({'cust_no': cust_no})

combined_train = pd.merge(init_df, cust_item_pair_train, on = 'cust_no', how = 'left')[['cust_no', 'item_id']]
# combined_train.item_id = combined_train.item_id.astype(int)
combined_test = pd.merge(init_df, cust_item_pair_test, on = 'cust_no', how = 'left')[['cust_no', 'item_id']]

In [9]:
def splitting(user_list, train_start, test_date):
    context_train = context[(context.date >= train_start) & (context.date < test_date)][['cust_no', 'item_id', 'show','click']]
    trainUniqueUsers = set(context_train.cust_no.unique())
    context_test = context[context.date == test_date][['cust_no', 'item_id','click']]
    context_test = context_test[context_test.cust_no.isin(trainUniqueUsers)]

    cust_item_pair_train = context_train.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
    cust_item_pair_test = context_test.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()

    cust_no = context_train.cust_no.unique()
    init_df = pd.DataFrame({'cust_no': cust_no})

    combined_train = pd.merge(init_df, cust_item_pair_train, on = 'cust_no', how = 'left')[['cust_no', 'item_id']]
    # combined_train.item_id = combined_train.item_id.astype(int)
    combined_test = pd.merge(init_df, cust_item_pair_test, on = 'cust_no', how = 'left')[['cust_no', 'item_id']]
    # combined_test.item_id = combined_test.item_id.replace(np.nan, -1)
    # combined_test = combined_test[combined_test.item_id != -1]
    # combined_test.item_id = combined_test.item_id.astype(int)
    
    items_by_cust_train = combined_train.groupby('cust_no')['item_id'].unique()
    items_by_cust_test = combined_test.groupby('cust_no')['item_id'].unique()

    cust_items_dict_train = dict(items_by_cust_train.apply(list))
    cust_items_dict_test = dict(items_by_cust_test.apply(list))
    return cust_items_dict_train, cust_items_dict_test

In [9]:
def export_file(date, pairs_dict_train, pairs_dict_test):
    dir_name = f'date={date}'
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
        
    ## train
    file_path = os.path.join(dir_name, 'train.txt')
    file = open(file_path,"w")
    print(f'write to file path: {file}')
    counts = 0
    for key,value in pairs_dict_train.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                else:
                    file.write(' %s' % int(item))
        file.write('\n')
        counts += 1
    print(counts)
    file.close()
    ## test
    file_path = os.path.join(dir_name, 'test.txt')
    file = open(file_path,"w")
    counts = 0
    print(f'write to file path: {file}')
    
    for key,value in pairs_dict_test.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                    file.write('\n')
                else:
                    file.write(' %s' % int(item))
        counts += 1
    print(counts)
    file.close()

In [ ]:
for test_date in range(20211226, 20211231+1):
    cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
    # export_file(date = str(test_date), pairs_dict_train = cust_items_dict_train, pairs_dict_test = cust_items_dict_test )

In [10]:
test_date = 20211226
cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
# export_file(date = str(test_date), pairs_dict_train = cust_items_dict_train, pairs_dict_test = cust_items_dict_test )
len_tr = len(list(cust_items_dict_train.keys()))
len_te = len(list(cust_items_dict_test.keys()))
print(len_tr)
print(len_te)
print('---')

450820
450820
---


In [11]:
cust_items_dict_test

{0: [1.0],
 1: [nan],
 2: [nan],
 3: [nan],
 4: [nan],
 5: [nan],
 8: [nan],
 9: [nan],
 10: [nan],
 11: [nan],
 13: [nan],
 14: [nan],
 15: [nan],
 17: [nan],
 22: [nan],
 24: [nan],
 25: [nan],
 26: [nan],
 27: [nan],
 28: [nan],
 29: [nan],
 30: [nan],
 32: [nan],
 33: [nan],
 34: [nan],
 35: [8.0],
 36: [nan],
 37: [nan],
 38: [nan],
 41: [nan],
 43: [nan],
 44: [nan],
 47: [nan],
 48: [nan],
 49: [nan],
 50: [nan],
 51: [nan],
 52: [nan],
 53: [nan],
 54: [nan],
 55: [nan],
 56: [nan],
 57: [nan],
 58: [nan],
 59: [nan],
 61: [nan],
 62: [nan],
 63: [9.0],
 64: [nan],
 65: [nan],
 66: [nan],
 67: [nan],
 68: [nan],
 69: [13.0],
 70: [nan],
 72: [nan],
 73: [9.0],
 75: [nan],
 77: [nan],
 78: [nan],
 79: [nan],
 80: [nan],
 81: [nan],
 82: [nan],
 83: [nan],
 84: [nan],
 86: [nan],
 87: [nan],
 88: [nan],
 89: [nan],
 90: [nan],
 91: [nan],
 92: [nan],
 93: [nan],
 94: [5.0],
 95: [nan],
 96: [nan],
 98: [nan],
 99: [0.0],
 100: [nan],
 101: [nan],
 102: [nan],
 105: [nan],
 106: [

In [9]:
for test_date in range(20211226, 20211231+1):
    cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
    export_file(date = str(test_date), pairs_dict_train = cust_items_dict_train, pairs_dict_test = cust_items_dict_test )
    len_tr = len(list(cust_items_dict_train.keys()))
    len_te = len(list(cust_items_dict_test.keys()))
    print(len_tr)
    print(len_te)
    print('---')

write to file path: <_io.TextIOWrapper name='date=20211226/train.txt' mode='w' encoding='UTF-8'>
450820
write to file path: <_io.TextIOWrapper name='date=20211226/test.txt' mode='w' encoding='UTF-8'>
54045
450820
54045
---
write to file path: <_io.TextIOWrapper name='date=20211227/train.txt' mode='w' encoding='UTF-8'>
471569
write to file path: <_io.TextIOWrapper name='date=20211227/test.txt' mode='w' encoding='UTF-8'>
89054
471569
89054
---
write to file path: <_io.TextIOWrapper name='date=20211228/train.txt' mode='w' encoding='UTF-8'>
499850
write to file path: <_io.TextIOWrapper name='date=20211228/test.txt' mode='w' encoding='UTF-8'>
99623
499850
99623
---
write to file path: <_io.TextIOWrapper name='date=20211229/train.txt' mode='w' encoding='UTF-8'>
528515
write to file path: <_io.TextIOWrapper name='date=20211229/test.txt' mode='w' encoding='UTF-8'>
98595
528515
98595
---
write to file path: <_io.TextIOWrapper name='date=20211230/train.txt' mode='w' encoding='UTF-8'>
553810
writ

In [12]:
# for test_date in range(20211226, 20211231+1):
#     cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
#     len_tr = len(list(cust_items_dict_train.keys()))
#     len_te = len(list(cust_items_dict_test.keys()))
#     print(len_tr)
#     print(len_te)
#     print('---')

8852
50
---
9606
105
---
10964
124
---
12372
114
---
13693
64
---
14486
47
---
